# Energy GDP and Publishing Data
This was an assignment as a part of a class I took on Coursera called _Introduction to Data Science in Python_ from the University of Michigan.

This assignment involved reading in 3 data sets, getting rid of missing/bad values in all of them, and merging certain portions of the data into a new data frame.

In [2]:
import pandas as pd
import numpy as np

## Reading and Cleaning Data

### Energy Data Set

Load data into a data frame from a `.xls` file skipping header and footer

In [5]:
xl = pd.ExcelFile("Energy Indicators.xls")
energy = xl.parse("Energy", skiprows=16, skip_footer=283-245)
energy.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Energy Supply,Energy Supply per capita,Renewable Electricity Production
0,NaN,NaN,NaN,Petajoules,Gigajoules,%
1,NaN,Afghanistan,Afghanistan,321,10,78.6693
2,NaN,Albania,Albania,102,35,100
3,NaN,Algeria,Algeria,1959,51,0.55101
4,NaN,American Samoa,American Samoa,...,...,0.641026


Remove first two columns and first row

In [6]:
energy = energy.drop(['Unnamed: 0', 'Unnamed: 1'], axis=1)
energy = energy.drop(0)
energy.head()

,Unnamed: 2,Energy Supply,Energy Supply per capita,Renewable Electricity Production
1,Afghanistan,321,10,78.6693
2,Albania,102,35,100
3,Algeria,1959,51,0.55101
4,American Samoa,...,...,0.641026
5,Andorra,9,121,88.6957


Rename columns

In [7]:
energy = energy.rename(index=int, columns={'Unnamed: 2' : 'Country', 'Renewable Electricity Production' : '% Renewable'})
energy.head()

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,321,10,78.6693
2,Albania,102,35,100
3,Algeria,1959,51,0.55101
4,American Samoa,...,...,0.641026
5,Andorra,9,121,88.6957


Set last three columns to a numeric type and convert missing values to `NaN`

In [10]:
energy['Energy Supply'] = pd.to_numeric(energy['Energy Supply'], errors='coerce')
energy['Energy Supply per capita'] = pd.to_numeric(energy['Energy Supply per capita'], errors='coerce')
energy['% Renewable'] = pd.to_numeric(energy['% Renewable'], errors='coerce')
energy.head()    

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,321.0,10.0,78.669280
2,Albania,102.0,35.0,100.000000
3,Algeria,1959.0,51.0,0.551010
4,American Samoa,NaN,NaN,0.641026
5,Andorra,9.0,121.0,88.695650


Convert `Energy Supply` to gigajoules

In [11]:
energy['Energy Supply'] = energy['Energy Supply']*1e6
energy.head()

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,3.210000e+08,10.0,78.669280
2,Albania,1.020000e+08,35.0,100.000000
3,Algeria,1.959000e+09,51.0,0.551010
4,American Samoa,NaN,NaN,0.641026
5,Andorra,9.000000e+06,121.0,88.695650


Take numbers and parentheses out of country names

In [14]:
energy['Country'] = energy['Country'].str.extract('(^[^()0-9]*)', expand=False)
energy.head()

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,3.210000e+08,10.0,78.669280
2,Albania,1.020000e+08,35.0,100.000000
3,Algeria,1.959000e+09,51.0,0.551010
4,American Samoa,NaN,NaN,0.641026
5,Andorra,9.000000e+06,121.0,88.695650


Rename some of the countries to match other data sets

In [16]:
energy['Country'] = energy['Country'].replace({"Republic of Korea": "South Korea",
                                  "United States of America": "United States",
                                  "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                  "China, Hong Kong Special Administrative Region": "Hong Kong"})

Set the index to `Country` for merging data sets

In [17]:
energy = energy.set_index('Country')
energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,3.210000e+08,10.0,78.669280
Albania,1.020000e+08,35.0,100.000000
Algeria,1.959000e+09,51.0,0.551010
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.695650


### GDP Data Set

Load data into data frame from a `.csv` file skipping the header

In [18]:
GDP = pd.read_csv('world_bank.csv', skiprows=4)
GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


Rename some of the countries to match the other data sets

In [21]:
GDP['Country Name'] = GDP['Country Name'].replace({"Korea, Rep.": "South Korea", 
                                                       "Iran, Islamic Rep.": "Iran",
                                                       "Hong Kong SAR, China": "Hong Kong"})

Set the index to Country for merging data sets

In [22]:
GDP = GDP.set_index('Country Name')
GDP.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


### Sciamgo Journal and Country Rank Data for Energy Engineering and Power Technology

Load data into data frame from an `.xlsx` file and set the index to be the country

In [23]:
xl2 = pd.ExcelFile("scimagojr-3.xlsx")
ScimEn = xl2.parse("Sheet1")
ScimEn = ScimEn.set_index('Country')

ScimEn.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Country,,,,,,,
China,1,127050,126767,597237,411683,4.70,138
United States,2,96661,94747,792274,265436,8.20,230
Japan,3,30504,30287,223024,61554,7.31,134
United Kingdom,4,20944,20357,206091,37874,9.84,139
Russian Federation,5,18534,18301,34266,12422,1.85,57


## Merging Data Sets

Choose only the countries that rank 15 or higher in the `ScimEn` data set, then merge with the data from `energy` for these countries

In [24]:
ScimEn_energy = pd.merge(ScimEn[ScimEn['Rank']  <= 15],energy, how='left', left_index=True, right_index=True)
ScimEn_energy.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable
Country,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868


Merge GDP data from `GDP` for those countries over the last 10 years

In [25]:
ScimEn_energy_GDP = pd.merge(ScimEn_energy, GDP[GDP.columns[-10:]], how='left', left_index=True, right_index=True)
ScimEn_energy_GDP

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
